In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from Managers.GameDirector import GameDirector
from Bots import RandomBot, AlexPastorBot, EdoBot, AdrianHerasBot
import pandas as pd
from tqdm import tqdm
import random

bots_configs = [
    [EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot, RandomBot.RandomBot],
    [EdoBot.EdoBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot],
    # [EdoBot.EdoBot, AdrianHerasBot.AdrianHerasBot, AdrianHerasBot.AdrianHerasBot, AdrianHerasBot.AdrianHerasBot],
    [EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot, RandomBot.RandomBot],
    [EdoBot.EdoBot, EdoBot.EdoBot, AlexPastorBot.AlexPastorBot, AlexPastorBot.AlexPastorBot],
    # [EdoBot.EdoBot, EdoBot.EdoBot, AdrianHerasBot.AdrianHerasBot, AdrianHerasBot.AdrianHerasBot],
    [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, RandomBot.RandomBot],
    [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, AlexPastorBot.AlexPastorBot],
    # [EdoBot.EdoBot, EdoBot.EdoBot, EdoBot.EdoBot, AdrianHerasBot.AdrianHerasBot],
    ]

def shuffle_bots(bots):
    names = [f"{bot.__name__}_{j}" for j, bot in enumerate(bots)]
    temp = list(zip(names, bots))
    random.shuffle(temp)
    res1, res2 = zip(*temp)
    names, bots = list(res1), list(res2)
    return names, bots

def play_single_game(bots):
    names, bots = shuffle_bots(bots)
    game_director = GameDirector(bots = bots)
    game_director.game_start(0)
    last_round = list(game_director.trace_loader.current_trace["game"].values())[-1]
    last_turn = list(last_round.values())[-1]
    points = list(last_turn["end_turn"]["victory_points"].values())
    points = [int(point) for point in points]
    return {name: points[i] for i, name in enumerate(names)}


def play_set_of_games(bots, games_to_play=100):
    return [play_single_game(bots) for i in tqdm(range(games_to_play))]

In [8]:
from joblib import delayed, Parallel

# results_list = Parallel(n_jobs=6)(delayed(play_set_of_games)(bots) for bots in bots_configs)
results_list = [ play_set_of_games(bots) for bots in bots_configs]

for results in results_list:
    df = pd.DataFrame(results)
    max_counts = dict(df.idxmax(axis=1).value_counts())
    bot_wins = {bot: max_counts.get(bot, 0)/len(df.index)*100 for bot in list(df.columns)}

    display(pd.DataFrame(bot_wins, index=["% wins"]).T)

 92%|█████████▏| 92/100 [00:12<00:00,  9.64it/s]

ValueError: empty range for randrange() (0, 0, 0)

 92%|█████████▏| 92/100 [00:28<00:00,  9.64it/s]